In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
import numpy as np


In [ ]:
import copy
from utils import resize_im

In [ ]:
im = cv2.imread('ow_niem.jpg')
im = cv2.resize(im, (224,224))
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))

In [ ]:
ff = 'db/low-resolution/1043-n000001-Shiba_Dog/n101045.jpeg'
# ff = 'db/low-resolution/1043-n000001-Shiba_Dog/n101040.jpeg'
ff = 'ow_niem.jpg'
im = cv2.imread(ff)
print(type(im))
print(im.shape)
im = resize_im(im, 224, True)
cv2.imwrite('aaa.png', im)
print(im.shape)
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
# cv2.imshow('1',im)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
print(tf.__version__)


In [ ]:
def get_class_from_folder(folder_name):
    return folder_name[folder_name.rindex('-')+1:].replace('_', ' ').capitalize()

def get_class_from_path(fname):
    s = fname[fname.rindex('-')+1:].replace('_', ' ').capitalize()
    return s[:s.index('/')] 


In [ ]:
dog_im_path = 'db/low-resolution'
dog_im_path_annot = 'db/Low-Annotations'

In [ ]:
#convert files with train file list
def convert_list_of_images(in_fname, out_fname):
    f = open(in_fname, 'r')
    lines = f.readlines()
    f.close()
    f = open(out_fname, 'w')
    for line in lines:
        f.write(line.strip()[3:]+'\n')
    f.close()

In [ ]:
ftrain = 'db/train.lst'
fval = 'db/validation.lst'

convert_list_of_images(ftrain, 'train_images.txt')
convert_list_of_images(fval, 'val_images.txt')

In [ ]:
class DogImageGeneratorSeq(Sequence):
    def __init__(self, path, fimlist, pathannot=None, cls_names = None, imsize = 224, imloop=False, batch_size=4, shuffle = True, to_fit = True, even_cls=False, prob_flip=0, prob_rot=0, rot_max = 20):
        self.path = path
        self.pathannot = pathannot
        self.fimlist = fimlist
        self.imsize = imsize
        self.imloop = imloop
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.to_fit = to_fit
        self.even_cls = even_cls
        self.prob_flip = prob_flip
        self.prob_rot = prob_rot
        self.rot_max = rot_max
        f = open(self.fimlist, 'r')
        self.ims = [x.strip() for x in f.readlines()]
        f.close()
        if cls_names:
            self.cls_names = cls_names
        else:
            cls_names = set()
            for x in self.ims:
                cls_names.add(get_class_from_path(x))
            cls_names = list(cls_names)
            cls_names.sort()
            self.cls_names = cls_names
        self.cls = []
        for x in self.ims:
            self.cls.append(self.cls_names.index(get_class_from_path(x)))
        self.indxs = np.arange(len(self.ims))
        self.on_epoch_end()
    def __len__(self):
        return int(np.floor(len(self.ims) / self.batch_size))
    def __getitem__(self, idx):
        if self.even_cls:
            return self.__getitem_even(idx)
        else:
            return self.__getitem__linear(idx)
    def __getitem__linear(self, idx):
        indxs_tmp = self.indxs[idx*self.batch_size:(idx+1)*self.batch_size]
        tmpX = [self._get_sample_(self.ims[i]) for i in indxs_tmp]
        tmpX = np.array(tmpX)
        tmpX = tmpX.astype(np.float32)/255
        tmpY = to_categorical([self.cls[i] for i in indxs_tmp], num_classes=len(self.cls_names))
        if self.to_fit:
            return tmpX, tmpY
        else:
            return tmpX            
    def __getitem_even(self, idx):
        tmpX = []
        tmpY = []
        for i in range(self.batch_size):
            ii = np.random.randint(len(self.cls_names))
            cims = [self.ims[idx] for idx in range(len(self.ims)) if self.cls[idx]==ii]
            imfile = cims[np.random.randint(len(cims))]
            tmpX.append(self._get_sample_(imfile))
            tmpY.append(ii)
        tmpX = np.array(tmpX)
        tmpX = tmpX.astype(np.float32)/255
        tmpY = to_categorical(tmpY, num_classes=len(self.cls_names))
        if self.to_fit:
            return tmpX, tmpY
        else:
            return tmpX
    def on_epoch_end(self):
        self.indxs = np.arange(len(self.ims))
        if self.shuffle:
            np.random.shuffle(self.indxs)
    def _get_sample_(self, impath):
        im = cv2.imread(self.path + '/' + impath)
        if self.pathannot: #get body annotation
            fxml = self.pathannot + '/' + impath + '.xml'
            tree = ET.parse(fxml)
            root = tree.getroot()
            box = root.find('object').find('bodybndbox')
            xmin, xmax, ymin, ymax = box.find('xmin').text, box.find('xmax').text, box.find('ymin').text, box.find('ymax').text
            xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
            im = im[ymin:ymax,xmin:xmax,:]
        if self.prob_flip > 0 and np.random.rand() < self.prob_flip:
            im = cv2.flip(im, [-1, 0, 1][np.random.randint(3)])
        if self.prob_rot > 0 and np.random.rand() < self.prob_rot:
            M = cv2.getRotationMatrix2D((im.shape[1]/2,im.shape[0]/2),-self.rot_max+np.random.rand()*2*self.rot_max,1)
            im = cv2.warpAffine(im, M, (im.shape[1], im.shape[0])) 
        im = resize_im(im, self.imsize, self.imloop)
        return im


In [ ]:
dgen = DogImageGeneratorSeq(dog_im_path, 'train_images.txt', pathannot=dog_im_path_annot, imloop=True, batch_size=4,
                            prob_flip=1, prob_rot=1, rot_max=20)
x, y = dgen[0]

plt.imshow(cv2.cvtColor(x[0], cv2.COLOR_BGR2RGB))

In [ ]:
tr_gen = DogImageGeneratorSeq(dog_im_path, 'train_images.txt', pathannot=dog_im_path_annot, cls_names = None, imsize = 224, imloop=True, batch_size=128, shuffle = True, to_fit = True, even_cls=True, prob_flip=0.5, prob_rot=0, rot_max = 20)
val_gen = DogImageGeneratorSeq(dog_im_path, 'val_images.txt', pathannot=None, cls_names = tr_gen.cls_names, imsize = 224, imloop=True, batch_size=128, shuffle = True, to_fit = True, even_cls=False)


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import load_model

In [ ]:
num_classes = len(tr_gen.cls_names)

imsize = tr_gen.imsize

model = Sequential()
model.add(Conv2D(32, 3, input_shape=(imsize, imsize, 3), activation='relu', data_format='channels_last'))
model.add(Conv2D(32, 5, activation='relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.5))
model.add(Conv2D(32, 3, activation='relu'))
model.add(Conv2D(32, 5, activation='relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.5))
model.add(Conv2D(32, 3, activation='relu'))
model.add(Conv2D(32, 5, activation='relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.5))
model.add(Conv2D(32, 3, activation='relu'))
model.add(Conv2D(32, 5, activation='relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model = load_model('_model_.hdf5')
model.summary()

logger = keras.callbacks.ModelCheckpoint('_model_.hdf5', monitor='val_accuracy', verbose=0, save_best_only=True)

In [ ]:

history = model.fit_generator(generator=tr_gen, validation_data=val_gen, epochs=100, verbose=1, callbacks=[logger], use_multiprocessing=False, workers=6)
print(history)

scores = model.evaluate(val_gen, verbose=1)
print("Validation error: %.2f%%" % (100-scores[1]*100))

#Wczytanie najlepszego modelu z pliku
model2 = load_model('_model_.hdf5')
# model2 = load_model('_model_.hdf5', compile=False)
# model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC()])
scores2 = model2.evaluate(val_gen, verbose=1)
print('model z pliku:')
print("validation error: %.2f%%" % (100-scores2[1]*100))

In [ ]:
# for RESNET
tr_gen = DogImageGeneratorSeq(dog_im_path, 'train_images.txt', pathannot=dog_im_path_annot, cls_names = None, imsize = 224, imloop=True, batch_size=32, shuffle = True, to_fit = True, even_cls=True, prob_flip=0.5, prob_rot=0.5, rot_max = 20)
val_gen = DogImageGeneratorSeq(dog_im_path, 'val_images.txt', pathannot=None, cls_names = tr_gen.cls_names, imsize = 224, imloop=True, batch_size=32, shuffle = True, to_fit = True, even_cls=False)

In [ ]:
# RESNET experiments
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(weights=None, include_top=False, input_shape=(224, 224, 3) )
# base_model.summary()
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(130, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
# freeze
# for lay in base_model.layers:
#     lay.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

logger = keras.callbacks.ModelCheckpoint('_model_.hdf5', monitor='val_accuracy', verbose=0, save_best_only=True)

history = model.fit_generator(generator=tr_gen, validation_data=val_gen, epochs=2, verbose=1, callbacks=[logger], use_multiprocessing=False, workers=6)
print(history)

scores = model.evaluate(val_gen, verbose=1)
print("Validation error: %.2f%%" % (100-scores[1]*100))

#Wczytanie najlepszego modelu z pliku
model2 = load_model('_model_.hdf5')
# model2 = load_model('_model_.hdf5', compile=False)
# model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC()])
scores2 = model2.evaluate(val_gen, verbose=1)
print('model z pliku:')
print("validation error: %.2f%%" % (100-scores2[1]*100))

In [ ]:
# resneet continue training
history = model.fit_generator(generator=tr_gen, validation_data=val_gen, epochs=48, verbose=1, callbacks=[logger], use_multiprocessing=False, workers=6)
print(history)

In [ ]:
model2 = load_model('_model_.hdf5')
scores_test = model2.evaluate(val_gen, verbose=2)
print("Test error: %.2f%%" % (100-scores_test[1]*100))

In [ ]:
val_gen2 = DogImageGeneratorSeq(dog_im_path, 'val_images.txt', pathannot=dog_im_path_annot, cls_names = tr_gen.cls_names, imsize = 224, imloop=True, batch_size=64, shuffle = True, to_fit = True, even_cls=False)
model2 = load_model('_model_.hdf5')
scores_test = model2.evaluate(val_gen2, verbose=2)
print("Test error: %.2f%%" % (100-scores_test[1]*100))

In [ ]:
tr_gen2 = DogImageGeneratorSeq(dog_im_path, 'train_images.txt', pathannot=dog_im_path_annot, cls_names = tr_gen.cls_names, imsize = 224, imloop=True, batch_size=64, shuffle = True, to_fit = True, even_cls=False)
model2 = load_model('_model_.hdf5')
scores_test = model2.evaluate(tr_gen2, verbose=2)
print("Test error: %.2f%%" % (100-scores_test[1]*100))

In [ ]:
#predykcja dla konkretnego obrazu
impath = 'images/bullterrier.jpg'
size = 224
im = cv2.imread(impath)
# plt.figure()
# plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
im = resize_im(im, size, True)
# plt.figure()
# plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
print(im.shape)

import numpy as np
im = np.expand_dims(im, axis=0)
preds = model2.predict(im).flatten()
# print(type(preds))
# print(preds)
print(preds.shape)
# print(preds)
indxs = preds.argsort()[::-1]
print(indxs[:10])
print(preds[indxs[:10]])
cls_names = np.array(val_gen2.cls_names)
print(cls_names[indxs[:10]])